# Monte Carlo Dispersion Analysis with the Dispersion Class

Finally the Monte Carlo Simulations can be performed using a dedicated class called Dispersion. This class is a wrapper for the Monte Carlo Simulations, and it is the recommended way to perform the simulations. Say goodbye to the long and tedious process of creating the Monte Carlo Simulations throughout jupyter notebooks!

In [1]:
%load_ext autoreload
%autoreload 2

First, let's import the necessary libraries, including the newest Dispersion class!

In [ ]:
from rocketpy import Environment, Rocket, SolidMotor, Flight, Dispersion


If you are using Jupyter Notebooks, it is recommended to run the following line to make matplotlib plots which will be shown later interactive and higher quality.

In [ ]:
%matplotlib inline

The Dispersion class allows us to perform Monte Carlo Simulations in a very simple way.
We just need to create an instance of the class, and then call the method run() to perform the simulations.
The class has a lot of capabilities, but we will only use a few of them in this example.
We encourage you to check the documentation of the class to learn more about the Dispersion.

Also, you can check RocketPy's main reference for a better conceptual understanding 
of the Monte Carlo Simulations: [RocketPy: Six Degree-of-Freedom Rocket Trajectory Simulator](https://doi.org/10.1061/(ASCE)AS.1943-5525.0001331)

We will describe two different options of usage:
- Using a Flight object as input, speeding up the process of creating the simulations.
- Using a dictionary as input, including mean values and uncertainties for each parameter 

You need only one of them to get started. 

## 1st Option -> Use your Flight object as input for the Dispersion class

### Creating an Environment for Ponte de Sôr, Portugal

In [ ]:
Env = Environment(
    railLength=5.2, latitude=39.389700, longitude=-8.288964, elevation=160
)


To get weather data from the GFS forecast, available online, we run the following lines.

First, we set tomorrow's date.

In [ ]:
import datetime

tomorrow = datetime.date.today() + datetime.timedelta(days=1)

Env.setDate((tomorrow.year, tomorrow.month, tomorrow.day, 12))  # Hour given in UTC time


Then, we tell Env to use a GFS forecast to get the atmospheric conditions for flight.

Don't mind the warning, it just means that not all variables, such as wind speed or atmospheric temperature, are available at all altitudes given by the forecast.

In [ ]:
Env.setAtmosphericModel(type="Forecast", file="GFS")


In [ ]:
Env.info()


### Creating a Motor for the Rocket

We define a motor for the rocket, using the data from the manufacturer, and following
the [RocketPy's documentation](https://docs.rocketpy.org/en/latest/user/index.html).

In [ ]:
Pro75M1670 = SolidMotor(
    thrustSource="data/motors/Cesaroni_M1670.eng",
    burnOut=3.9,
    grainNumber=5,
    grainSeparation=5 / 1000,
    grainDensity=1815,
    grainOuterRadius=33 / 1000,
    grainInitialInnerRadius=15 / 1000,
    grainInitialHeight=120 / 1000,
    nozzleRadius=33 / 1000,
    throatRadius=11 / 1000,
    interpolationMethod="linear",
)


In [ ]:
Pro75M1670.info()


### Creating a Rocket

In [ ]:
Calisto = Rocket(
    motor=Pro75M1670,
    radius=127 / 2000,
    mass=19.197 - 2.956,
    inertiaI=6.60,
    inertiaZ=0.0351,
    distanceRocketNozzle=-1.255,
    distanceRocketPropellant=-0.85704,
    powerOffDrag="data/calisto/powerOffDragCurve.csv",
    powerOnDrag="data/calisto/powerOnDragCurve.csv",
)

Calisto.setRailButtons([0.2, -0.5])

NoseCone = Calisto.addNose(length=0.55829, kind="vonKarman", distanceToCM=0.71971)

FinSet = Calisto.addFins(
    4, span=0.100, rootChord=0.120, tipChord=0.040, distanceToCM=-1.04956
)

Tail = Calisto.addTail(
    topRadius=0.0635, bottomRadius=0.0435, length=0.060, distanceToCM=-1.194656
)


Additionally, we set parachutes for our Rocket, as well as the trigger functions for the deployment of such parachutes.

In [ ]:
def drogueTrigger(p, y):
    # p = pressure
    # y = [x, y, z, vx, vy, vz, e0, e1, e2, e3, w1, w2, w3]
    # activate drogue when vz < 0 m/s.
    return True if y[5] < 0 else False


def mainTrigger(p, y):
    # p = pressure
    # y = [x, y, z, vx, vy, vz, e0, e1, e2, e3, w1, w2, w3]
    # activate main when vz < 0 m/s and z < 800 + 1400 m (+1400 due to surface elevation).
    return True if y[5] < 0 and y[2] < 800 + 1400 else False


Main = Calisto.addParachute(
    "Main",
    CdS=10.0,
    trigger=mainTrigger,
    samplingRate=105,
    lag=1.5,
    noise=(0, 8.3, 0.5),
)

Drogue = Calisto.addParachute(
    "Drogue",
    CdS=1.0,
    trigger=drogueTrigger,
    samplingRate=105,
    lag=1.5,
    noise=(0, 8.3, 0.5),
)


In [ ]:
Calisto.info()


### Simulate single flight

In [ ]:
TestFlight = Flight(rocket=Calisto, environment=Env, inclination=84, heading=133)


And we can visualize the flight trajectory:

In [ ]:
TestFlight.plot3DTrajectory()

Export Flight Trajectory to a .kml file so it can be opened on Google Earth

### Starting the Monte Carlo Simulations

First, let's invoke the Dispersion class, we only need a filename to initialize it.
The filename will be used either to save the results of the simulations or to load them
from a previous ran simulation.

In [ ]:
TestDispersion = Dispersion(filename="data/docs/dispersion_analysis/")

Then, we can run the simulations using the method Dispersion.run_dispersion().
But before that, we need to set some simple parameters for the simulations.
We will set them by using a dictionary, which is one of the simplest way to do it.

In [ ]:
# TODO: Fill this with the correct values and the missed ones
dispersionDict = {
    "rocketMass": (19.197 - 2.956, 0.2),
    "radius": 0.1 * 127 / 2000,
    "burnOut": (3.9, 0.5),
    "parachuteNames": ["Main", "Drogue"],
    "CdS": [(10, 2), (1)],
    "trigger": [[mainTrigger], [drogueTrigger]],
}

Finally, let's iterate over the simulations and export the data from each flight simulation!

In [ ]:
TestDispersion.runDispersion(
    number_of_simulations=10, # Be careful with this number, it will take a while to run
    dispersionDict=dispersionDict,
    flight=TestFlight,
)

RocketPy separates the running of simulations from the importing of data.
This is done to allow the user to run simulations in parallel, if desired, and
to allow the user to import data from different sources.

Remember to specify a good name for your loaded data, so you can easily identify
and treat them later. 

In [ ]:
dispersion_results = TestDispersion.import_results()

### Visualizing the results

Now we finally have the results of our Monte Carlo simulations loaded!
Let's play with them.

First, we can print numerical information regarding the results of the simulations.

In [ ]:
TestDispersion.info()

Also, we can visualize histograms of such results

In [ ]:
TestDispersion.allInfo()

And finally, we can export the ellipses of the results to a .kml file so it can be opened on Google Earth

In [ ]:
TestDispersion.exportEllipsesToKML(
    dispersion_results,
    filename,
    origin_lat,
    origin_lon,
    type="all",
    resolution=100,
    color="ff0000ff",
)

## 2nd Option -> Running by using only a dictionary of parameters

This second option allow us to perform the Monte Carlo Simulations without the need of a Flight object. This is useful when we want to perform the simulations for a rocket that we don't have a Flight object for, or when we want to perform the simulations for a rocket that we have a Flight object for, but we want to change some parameters of the simulations.

In [ ]:
analysis_parameters = {
    # Mass Details
    "mass": (
        8.257,
        0.001,
    ),  # Rocket's dry mass (kg) and its uncertainty (standard deviation)
    # Propulsion Details - run help(SolidMotor) for more information
    # "thrust": ["docs/notebooks/dispersion_analysis/dispersion_analysis_inputs/thrustCurve.csv"],
    # "totalImpulse": (1415.15, 35.3),  # Motor total impulse (N*s)
    # "burnOutTime": (5.274, 1),  # Motor burn out time (s)
    # "nozzleRadius": (21.642 / 1000, 0.5 / 1000),  # Motor's nozzle radius (m)
    # "throatRadius": (8 / 1000, 0.5 / 1000),  # Motor's nozzle throat radius (m)
    # "grainNumber":[6],
    # "grainSeparation": (
    #     6 / 1000,
    #     1 / 1000,
    # ),  # Motor's grain separation (axial distance between two grains) (m)
    # "grainDensity": (1707, 50),  # Motor's grain density (kg/m^3)
    # "grainOuterRadius": (21.4 / 1000, 0.375 / 1000),  # Motor's grain outer radius (m)
    # "grainInitialInnerRadius": (
    #     9.65 / 1000,
    #     0.375 / 1000,
    # ),  # Motor's grain inner radius (m)
    # "grainInitialHeight": (120 / 1000, 1 / 1000),  # Motor's grain height (m)
    # "interpolationMethod":["linear"],
    # Aerodynamic Details - run help(Rocket) for more information
    "inertiaI": (
        3.675,
        0.03675,
    ),  # Rocket's inertia moment perpendicular to its axis (kg*m^2)
    "inertiaZ": (
        0.007,
        0.00007,
    ),  # Rocket's inertia moment relative to its axis (kg*m^2)
    "radius": (40.45 / 1000, 0.001),  # Rocket's radius (kg*m^2)
    "distanceRocketNozzle": (
        -1.024,
        0.001,
    ),  # Distance between rocket's center of dry mass and nozzle exit plane (m) (negative)
    "distanceRocketPropellant": (
        -0.571,
        0.001,
    ),  # Distance between rocket's center of dry mass and and center of propellant mass (m) (negative)
    "powerOffDrag": (
        0.9081 / 1.05,
        0.033,
    ),  # Multiplier for rocket's drag curve. Usually has a mean value of 1 and a uncertainty of 5% to 10%
    "powerOnDrag": (
        0.9081 / 1.05,
        0.033,
    ),  # Multiplier for rocket's drag curve. Usually has a mean value of 1 and a uncertainty of 5% to 10%
    "noseKind": ["Von Karman"],
    "noseLength": (0.274, 0.001),  # Rocket's nose cone length (m)
    "noseDistanceToCM": (
        1.134,
        0.001,
    ),  # Axial distance between rocket's center of dry mass and nearest point in its nose cone (m)
    "numberOfFins": [4],
    "span": (0.077, 0.0005),  # Fin span (m)
    "rootChord": (0.058, 0.0005),  # Fin root chord (m)
    "tipChord": (0.018, 0.0005),  # Fin tip chord (m)
    "distanceToCM": (
        -0.906,
        0.001,
    ),  # Axial distance between rocket's center of dry mass and nearest point in its fin (m)
    "airfoil": [None],
    "noTail": [True],
    "positionFirstRailButton": [0.0224],
    "positionSecondRailButton": [-0.93],
    "railButtonAngularPosition": [30],
    # Launch and Environment Details - run help(Environment) and help(Flight) for more information
    "inclination": (
        84.7,
        1,
    ),  # Launch rail inclination angle relative to the horizontal plane (degrees)
    "heading": (53, 2),  # Launch rail heading relative to north (degrees)
    "railLength": (5.7, 0.0005),  # Launch rail length (m)
    "ensembleMember": list(range(10)),  # Members of the ensemble forecast to be used
    # Parachute Details - run help(Rocket) for more information
    "parachuteNames": ["Drogue"],
    "CdS": [
        (
            0.349 * 1.3,
            0.07,
        )
    ],  # Drag coefficient times reference area for the drogue chute (m^2)
    "trigger": [[drogueTrigger]],
    "lag": [
        (
            173,
            0.5,
        )
    ],  # Time delay between parachute ejection signal is detected and parachute is inflated (s)
    "samplingRate": [[105]],
    "noise_mean": [[0]],
    "noise_sd": [[8.3]],
    "noise_corr": [[0.5]],
}


In [ ]:
# motorDisp = SolidMotor(
#     thrustSource="docs/notebooks/dispersion_analysis/dispersion_analysis_inputs/thrustCurve.csv",
#     burnOut=5.274,
#     grainNumber=6,
#     grainDensity=1707,
#     grainOuterRadius=21.4 / 1000,
#     grainInitialInnerRadius=9.65 / 1000,
#     grainInitialHeight=120 / 1000,
#     grainSeparation=6 / 1000,
#     nozzleRadius=21.642 / 1000,
#     throatRadius=8 / 1000,
#     reshapeThrustCurve=False,
#     interpolationMethod="linear",
# )


In [ ]:
# Disp = Dispersion("test")
# modified_dispersion_dict = {i: j for i, j in analysis_parameters.items()}
# Disp.motor = motorDisp
# Disp.processDispersionDict(modified_dispersion_dict)


In [ ]:
# Disp = Dispersion("test")
# Disp.runDispersion(100, analysis_parameters, Env, motor=motorDisp)
